# استراد المكتابات 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Miscellaneous
from tqdm import tqdm
import random
import os
import time
# Turn off tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# For Data Processing & ML Models
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image, ImageEnhance
import cv2

# For Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Enable offline usage of plotly
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

from IPython.display import clear_output
clear_output()

In [2]:
class cfg:
    IMAGE_SIZE = 128

    # Data Augmentation
    BRIGHTNESS = (0.64, 1.37) # (MIN, MAX)
    CONTRAST   = (0.64, 1.37) # (MIN, MAX)

    # Train-Val Split
    TRAIN_VAL_SPLIT = 0.7 # 80% of all the samples are used for training, and the rest for validation

    # Model Training
    BATCH_SIZE = 2
    EPOCHS = 1
    LEARNING_RATE = 0.0001

#   TensorFlow تحميل الصور وتجهيزها للاستخدام في 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# تحديد مسارات مجموعات البيانات
train_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\train'
validation_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\val'
test_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\test'

# تحديد الحجم والدفعة لعملية التحميل
img_width, img_height = 150, 150
batch_size = 32

# تحديد مولدات الصور لكل مجموعة
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
tf.random.set_seed(100)
np.random.seed(100)

tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
data_directories = ["C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\test\\",
                    "C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\train\\",
                    "C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\val\\"]

all_paths = []
all_labels = []

for data_dir in data_directories:
    for label in os.listdir(data_dir):
        for image in os.listdir(data_dir+label):
            all_paths.append(data_dir+label+'/'+image)
            if label=='TURBERCULOSIS':
                all_labels.append('TUBERCULOSIS')
            else:
                all_labels.append(label)

all_paths, all_labels = shuffle(all_paths, all_labels)

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({'path':all_paths, 'label':all_labels})

In [ ]:
from sklearn.utils import shuffle
import pandas as pd
# هنا يتم خلط all_paths و all_labels
all_paths, all_labels = shuffle(all_paths, all_labels)

data_directories = ["C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\test\\",
                    "C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\train\\",
                    "C:\\Users\\ibrahim IA\\Desktop\\XRay_1\\val\\"]

all_paths = []
all_labels = []

for data_dir in data_directories:
    for label in os.listdir(data_dir):
        for image in os.listdir(data_dir+label):
            all_paths.append(data_dir+label+'/'+image)
            if label=='TURBERCULOSIS':
                all_labels.append('TURBERCULOSIS')
            else:
                all_labels.append(label)

all_paths, all_labels = shuffle(all_paths, all_labels)

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({'path':all_paths, 'label':all_labels})
df


# رسم بيانات

In [ ]:
import matplotlib.pyplot as plt

# بيانات التدريب والاختبار
train_data = [6376, 1245, 1694]  
labels = ['Training', 'Validation', 'Testing']

plt.figure(figsize=(8, 6))
plt.bar(labels, train_data, color=['blue', 'orange', 'green'])
plt.xlabel('Data Types')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Different Sets')
plt.show()



# عرض بعض الإحصائيات حول بيانات الصور وعرض أمثلة منها

In [ ]:
plt.rcParams.update({'font.size': 13})

plt.figure(figsize=(7,7))

# plotting data on chart
plt.pie(list(df.label.value_counts()),
        labels=list(df.label.value_counts().index),
        colors=['#43b0c1', '#368d9a', '#286a74', '#69c0cd'],
        autopct='%.0f%%', wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' })

my_circle=plt.Circle((0,0), 0.675, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('Dataset\nDistribution', x=0.5, y=0.45) 
plt.show()

In [ ]:
def augment_image(image):
    # Convert input image from numpy to PIL Image
    image = Image.fromarray(np.uint8(image)) 
    # Select brightness and contrast factors
    brightness_factor = random.uniform(cfg.BRIGHTNESS[0],cfg.BRIGHTNESS[1])
    contrast_factor = random.uniform(cfg.CONTRAST[0],cfg.CONTRAST[1])
    # Apply Transformations
    image = ImageEnhance.Brightness(image).enhance(brightness_factor)
    image = ImageEnhance.Contrast(image).enhance(contrast_factor)
    return image

In [ ]:
def open_images(paths, augment=True):
    '''
    Given a list of paths to images, this function returns the images as arrays, and conditionally augments them
    '''
    images = []
    for path in paths:
        image = load_img(path, target_size=(cfg.IMAGE_SIZE,cfg.IMAGE_SIZE))
        if augment:
            image = augment_image(image)
        image = np.array(image)
        image = image/image.max()
        images.append(image)
    return np.array(images)

# Randomly select and plot a few images with augmentation
k = random.randint(0,5000)
image_paths = list(df.path[k:k+10])
labels = list(df.label[k:k+10])
images = open_images(image_paths, augment=True)

plt.rcParams.update({'font.size': 10})
fig = plt.figure(figsize=(20, 8))

for i in range(0, 10):
    fig.add_subplot(2, 5, i+1)
    plt.imshow(images[i])
    plt.axis('off')
    plt.title(labels[i])
plt.show()

In [ ]:
LABELS = ['NORMAL', 'TURBERCULOSIS', 'PNEUMONIA', 'COVID19']
label_encoder = {'NORMAL': 0, 'TURBERCULOSIS': 1, 'PNEUMONIA': 2, 'COVID19': 3}
label_decoder = {0: 'NORMAL', 1: 'TURBERCULOSIS', 2: 'PNEUMONIA', 3: 'COVID19'}

In [ ]:
import matplotlib.pyplot as plt

# عرض بعض الصور من مجموعة البيانات
def plot_images(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# عرض بعض الصور من مجموعة التدريب
sample_training_images, _ = next(train_generator)
plot_images(sample_training_images[:10]) 


# نموذج CNN & VGG16 باستخدام TensorFlow/Keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# تعيين مسارات البيانات
train_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay1\train'
validation_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\val'
test_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\test'

# تحديد الأبعاد
img_width, img_height = 224, 224
batch_size = 32
num_classes = 4

# إعداد المولدات
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# تحضير البيانات
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# تحميل نموذج VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# بناء النموذج
model = Sequential()
model.add(vgg_base)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# تجميد طبقات VGG16
for layer in vgg_base.layers:
    layer.trainable = False

# تحديد الخوارزمية والتدريب
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# تقييم النموذج
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')


# عملية التدريب

In [ ]:
df_train = df[:int(cfg.TRAIN_VAL_SPLIT*len(df))]
df_val = df[int(cfg.TRAIN_VAL_SPLIT*len(df)):]

In [ ]:
plt.rcParams.update({'font.size': 13})
fig = plt.figure(figsize=(14, 7))

fig.add_subplot(1, 2, 1)
plt.pie(list(df_train.label.value_counts()),
        labels=list(df_train.label.value_counts().index),
        colors=['#43b0c1', '#368d9a', '#286a74', '#69c0cd'],
        autopct='%.0f%%', wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' })

my_circle=plt.Circle((0,0), 0.675, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.axis('off')
plt.title('Training', x=0.5, y=0.5) 


fig.add_subplot(1, 2, 2)
plt.pie(list(df_val.label.value_counts()),
        labels=list(df_val.label.value_counts().index),
        colors=['#43b0c1', '#368d9a', '#286a74', '#69c0cd'],
        autopct='%.0f%%', wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' })

my_circle=plt.Circle((0,0), 0.675, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.axis('off')
plt.title('Validation', x=0.5, y=0.5) 

plt.show()

In [ ]:
def data_generator(df, batch_size=cfg.BATCH_SIZE, augment=True, epochs=cfg.EPOCHS):
    for e in range(epochs):
        for x in range(0,len(df), batch_size):
            image_paths = df.path[x:x+batch_size]
            images = open_images(image_paths, augment=augment)
            labels = df.label[x:x+batch_size]
            labels = [label_encoder[label] for label in labels]
            yield images, np.array(labels)

In [ ]:
# Creating generators for training and validation. You can change the parameters in the configuration section.
train_data_generator = data_generator(df_train, batch_size=cfg.BATCH_SIZE, augment=True, epochs=cfg.EPOCHS)
train_steps = int(len(df_train)/cfg.BATCH_SIZE)

val_data_generator = data_generator(df_val, batch_size=cfg.BATCH_SIZE, augment=False, epochs=cfg.EPOCHS)
val_steps = int(len(df_val)/cfg.BATCH_SIZE)

# النموذج ( model )

In [ ]:
model = models.Sequential(name='classifier')
model.add(layers.Input(shape=(cfg.IMAGE_SIZE, cfg.IMAGE_SIZE, 3), name='input'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name='block1_conv1'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name='block1_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool1'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv1'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool2'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv1'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool3'))
model.add(layers.Flatten(name='flatten'))
model.add(layers.Dropout(0.3, name='dropout1'))
model.add(layers.Dense(128, activation='relu', name='dense1'))
model.add(layers.Dense(4, activation='softmax', name='final'))

model.compile(optimizer=optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])


In [ ]:
from tensorflow.keras import models, layers, optimizers

# تعيين مسارات البيانات
train_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay1\train'
validation_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\val'
test_data_dir = r'C:\Users\ibrahim IA\Desktop\XRay_1\test'

# تحديد الأبعاد
img_width, img_height = 150, 150
batch_size = 32
num_classes = 4


# تحسينات على الشبكة العصبية
model = models.Sequential(name='classifier')
model.add(layers.Input(shape=(cfg.IMAGE_SIZE, cfg.IMAGE_SIZE, 3), name='input'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name='block1_conv1'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name='block1_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool1'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv1'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool2'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv1'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv2'))
model.add(layers.MaxPool2D(pool_size=(2,2), name='pool3'))
model.add(layers.Flatten(name='flatten'))
model.add(layers.Dropout(0.5, name='dropout1'))  # زيادة نسبة الإسقاط
model.add(layers.Dense(256, activation='relu', name='dense1'))  # زيادة عدد الوحدات
model.add(layers.Dropout(0.3, name='dropout2'))  # إضافة إسقاط إضافي
model.add(layers.Dense(4, activation='softmax', name='final'))

# Model compilation
model.compile(optimizer=optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

class cfg:
    IMAGE_SIZE = 128

    # Data Augmentation
    BRIGHTNESS = (0.64, 1.37) # (MIN, MAX)
    CONTRAST   = (0.64, 1.37) # (MIN, MAX)

    # Train-Val Split
    TRAIN_VAL_SPLIT = 0.7 # 80% of all the samples are used for training, and the rest for validation

    # Model Training
    BATCH_SIZE = 2
    EPOCHS = 1
    LEARNING_RATE = 0.001

In [ ]:
class_weights = (df_train.label.value_counts() / len(df_train)).to_dict()
class_weights = dict((label_encoder[key], value) for (key, value) in class_weights.items())
class_weights
class_weights[2] = 0.6  # تغيير قيمة الفئة 2 إلى 0.6
class_weights[0] = 0.2  # تغيير قيمة الفئة 0 إلى 0.2
class_weights[1] = 0.1  # تغيير قيمة الفئة 1 إلى 0.1
class_weights[3] = 0.1  # تغيير قيمة الفئة 3 إلى 0.1


#  الدقة (Accuracy)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_data_dir = r'C:/Users/ibrahim IA/Desktop/XRay_1/test'     

# تحديد حجم الصور والدفعة
img_width, img_height = 150, 150
batch_size = 32

# إعداد مولِّد البيانات لمجموعة الاختبار
test_datagen = ImageDataGenerator(rescale=1.0/255)  
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=False  
)

# التنبؤ بالتسميات باستخدام النموذج
predictions = model.predict(test_generator)

# الحصول على تسميات الفئات الحقيقية
true_labels = test_generator.classes

# تقييم الأداء، يمكنك استخدام مكتبة scikit-learn لحساب دقة النموذج
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, np.argmax(predictions, axis=-1))
print("Accuracy:", accuracy)



# مصفوفة الارتباط (Confusion Matrix)

In [ ]:
# استيراد المكتبة
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# حساب مصفوفة الارتباط
conf_matrix = confusion_matrix(true_labels, predictions.argmax(axis=-1))

# عرض مصفوفة الارتباط باستخدام heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


#  معدلات الاستدعاء والدقة (Recall and Precision)

In [ ]:
# حساب معدل الاستدعاء والدقة
from sklearn.metrics import recall_score, precision_score

# حساب معدل الاستدعاء والدقة
recall = recall_score(true_labels, predictions.argmax(axis=-1), average='weighted')
precision = precision_score(true_labels, predictions.argmax(axis=-1), average='weighted')

print(f'Recall: {recall}')
print(f'Precision: {precision}')



#  مقياس F1 (F1 Score)

In [ ]:
# حساب مقياس F1
from sklearn.metrics import f1_score

# حساب مقياس F1
f1 = f1_score(true_labels, predictions.argmax(axis=-1), average='weighted')
print(f'F1 Score: {f1}')


# عرض الدقة والخسارة من تاريخ التدريب

In [ ]:
import matplotlib.pyplot as plt

# دقة التدريب والتحقق
plt.plot(history.history['accuracy'], label='Accuracy (Training)')
plt.plot(history.history['val_accuracy'], label='Accuracy (Validation)')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# خسارة التدريب والتحقق
plt.plot(history.history['loss'], label='Loss (Training)')
plt.plot(history.history['val_loss'], label='Loss (Validation)')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



# كيفية تقييم أداء النموذج باستخدام بيانات الاختبار

In [ ]:
val_data_generator = data_generator(df_val, batch_size=cfg.BATCH_SIZE, augment=False, epochs=1)
val_steps = int(len(df_val)/cfg.BATCH_SIZE)

y_pred = []
y_true = []

for x,y in tqdm(val_data_generator, total=val_steps):
    pred = model.predict(x)
    pred = np.argmax(pred, axis=-1)
    for i in pred:
        y_pred.append(label_decoder[i])
    for i in y:
        y_true.append(label_decoder[i])
clear_output()
print(classification_report(y_true, y_pred, digits=4))

In [ ]:
val_data_generator = data_generator(df_val, batch_size=cfg.BATCH_SIZE, augment=False, epochs=1)
val_steps = int(len(df_val) / cfg.BATCH_SIZE)

y_pred = []
y_true = []

for x, y in tqdm(val_data_generator, total=val_steps):
    pred = model.predict(x)
    pred = np.argmax(pred, axis=-1)
    
    # Print true and predicted labels for inspection
    print("True labels:", [label_decoder[i] for i in y])
    print("Predicted labels:", [label_decoder[i] for i in pred])
    
    y_pred.extend(pred)
    y_true.extend(y)

clear_output()
print(classification_report(y_true, y_pred, digits=4))


In [ ]:
def predict(image, model):
    '''
    Input the image and model, this function outputs the prediction as:
        1. The class with the highest probability
        2. A dictionary containing each class with their corresponding probability
    '''
    labels = ['NORMAL', 'TURBERCULOSIS', 'PNEUMONIA', 'COVID19']
    image = np.array(image)
    image = image/image.max()
    image = image.reshape(-1,cfg.IMAGE_SIZE,cfg.IMAGE_SIZE,3)
    probabilities = model.predict(image).reshape(-1)
    pred = labels[np.argmax(probabilities)]
    return pred, {x:y for x,y in zip(labels, probabilities)}

INDICES = random.sample(list(df.index), 7)

for INDEX in INDICES:

    image = load_img(df.path[INDEX], target_size=(cfg.IMAGE_SIZE, cfg.IMAGE_SIZE))
    pred, probabilities = predict(image, model)

    x = list(probabilities.keys())
    y = list(probabilities.values())

    sns.set_style("darkgrid")
    fig, ax = plt.subplots(1,2, figsize=(16,4), gridspec_kw={'width_ratios': [3, 4]})
    ax[0].imshow(image)
    ax[0].axis('off')

    bars = ax[1].barh(x, y, height=0.3, color=['#69c0cd', '#69c0cd', '#69c0cd', '#69c0cd'])
    ax[1].bar_label(bars)
    
    ax[1].set_title('Predicted Class Probabilities')
    plt.xlim([0, 1])
    plt.show()

    print('Actual   :',df.label[INDEX])
    print('Predicted:', pred)
    print('-'*80)
    print('\n')

In [ ]:
val_data_generator = data_generator(df_val, batch_size=cfg.BATCH_SIZE, augment=False, epochs=1)
val_steps = int(len(df_val)/cfg.BATCH_SIZE)

y_pred = []
y_true = []

for x,y in tqdm(val_data_generator, total=val_steps):
    pred = model.predict(x)
    pred = np.argmax(pred, axis=-1)
    for i in pred:
        y_pred.append(label_decoder[i])
    for i in y:
        y_true.append(label_decoder[i])
clear_output()
print(classification_report(y_true, y_pred, digits=4))